In [ ]:
# Run in terminal!
# vllm serve "ConfidentialMind/Mistral-Small-24B-Instruct-2501_GPTQ_G32_W4A16"

In [18]:
from openai import OpenAI
import os
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch

In [19]:
# Load BLIP-2
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

In [40]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from PIL import Image
import torch

# Use the smaller BLIP-2 model with OPT
model_id = "Salesforce/blip2-opt-2.7b"
processor = Blip2Processor.from_pretrained(model_id)
model = Blip2ForConditionalGeneration.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

In [44]:
# Load image
image = Image.open("jeans.png").convert("RGB")

# Use instruction-style prompt as a QUESTION (not declarative)
prompt = "What is the person wearing? Describe color, fabric, and style."

# Process
inputs = processor(image, prompt, return_tensors="pt").to("cpu")

# Generate
with torch.no_grad():
    generated_ids = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=False,
        temperature=0.7
    )

# Decode
caption = processor.decode(generated_ids[0], skip_special_tokens=True)
print("🧵 Caption:", caption)

🧵 Caption: What is the person wearing? Describe color, fabric, and style.



In [47]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch

model_id = "Salesforce/blip-image-captioning-base"
processor = BlipProcessor.from_pretrained(model_id)
model = BlipForConditionalGeneration.from_pretrained(model_id).to("cpu")  # force CPU

image = Image.open("jeans.png").convert("RGB")
prompt = ""  # <-- this is key

inputs = processor(image, prompt, return_tensors="pt").to("cpu")
outputs = model.generate(**inputs, max_length=50)
caption = processor.decode(outputs[0], skip_special_tokens=True)

print("Caption:", caption)

Caption: describe primary clothing item. be descriptive pants pants pants pants pants pants pants pants pants pants pants pants pants pants pants pants pants


In [38]:
# Light-weight BLIP model (needs ~2GB GPU)
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch

model_id = "Salesforce/blip-image-captioning-base"  # or blip-image-captioning-large
processor = BlipProcessor.from_pretrained(model_id)
model = BlipForConditionalGeneration.from_pretrained(model_id).to("cpu")#to("cuda" if torch.cuda.is_available() else "cpu")

image = Image.open("jeans.png").convert("RGB")
prompt = "Describe the primary piece of clothing. Describe colors, patterns, fabrics, and styles."

inputs = processor(image, prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_length=50)
print(processor.decode(outputs[0], skip_special_tokens=True))

describe the primary piece of clothing. describe colors, patterns, fabrics, and styles.s


In [24]:
# Point to your local VLLM server
client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="sk-fake-key"  # Any string works; vLLM doesn’t check this
)

In [48]:
message = f'Return just the description of the article of clothing: {caption}. Return no other text.'

response = client.chat.completions.create(
    model="ConfidentialMind/Mistral-Small-24B-Instruct-2501_GPTQ_G32_W4A16",
    messages=[
        {"role": "user", "content": message}
    ],
    temperature=0.15
)

# Print the response
print(response.choices[0].message.content)

TypeError: Object of type Image is not JSON serializable